In [35]:
import csv
import numpy

In [27]:
path_pm25 = 'C:/Users/Ian/Documents/PythonDataProducts4PredictiveAnalytics/DesignThinking&PredictiveAnalytics4DataProducts/Final_Course2/datasets/PRSA_data_2010.1.1-2014.12.31.csv'
file = open(path, 'r')

In [28]:
dataset = csv.reader(file)
header = next(dataset)
header

['No',
 'year',
 'month',
 'day',
 'hour',
 'pm2.5',
 'DEWP',
 'TEMP',
 'PRES',
 'cbwd',
 'Iws',
 'Is',
 'Ir']

Obtenemos el índice de las etiquetas de interés

In [29]:
print(header.index('pm2.5'))
print(header.index('TEMP'))

5
7


Construcción del vector y de etiquetas

In [30]:
dataset = [d for d in dataset if d[5] != 'NA']
y = [float(d[5]) for d in dataset]

Construcción de la matriz X de características. A esta función se le pueden agregar más características además de la temperatura

In [32]:
def feature(datnum):
    feat = [1, float(datnum[7])]
    return feat

X = [feature(d) for d in dataset]

Ejemplo de los datos

In [34]:
print(y[:10])
print(X[:10])

[129.0, 148.0, 159.0, 181.0, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0]
[[1, -4.0], [1, -4.0], [1, -5.0], [1, -5.0], [1, -5.0], [1, -6.0], [1, -6.0], [1, -5.0], [1, -6.0], [1, -5.0]]


Recordemos que la expresión queda de la siguiente manera  
$pm2.5 = \theta_0 + \theta_1 * temp$

In [36]:
theta, residuals, rank, s = numpy.linalg.lstsq(X,y)
print(theta)

[107.10183392  -0.68447989]


<ipython-input-36-1e07fe70644f>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, residuals, rank, s = numpy.linalg.lstsq(X,y)


Por lo tanto, la línea de regresión resulta ser la siguiente  
$pm2.5 = 107.10183392 - 0.68447989 * temp$

Realizamos las operaciones de forma manual utilizando la expresión  
$ (X^TX)^{-1} X^Ty$

In [39]:
X_matrix = numpy.matrix(X)
y_matrix = numpy.matrix(y)
numpy.linalg.inv(X_matrix.T * X_matrix) * X_matrix.T * y_matrix.T

matrix([[107.10183392],
        [ -0.68447989]])